# Part B

## step 1 - Load the data and add fields to it

### Load the data

In [ ]:
import pandas as pd


file_path = "./hotels_data.csv"  
df = pd.read_csv(file_path)


df.head()


### Add columns and load to new CSV

In [ ]:
#Ensure dates are in correct format
df['Snapshot Date'] = pd.to_datetime(df['Snapshot Date'])
df['Checkin Date'] = pd.to_datetime(df['Checkin Date'])


df['DayDiff'] = (df['Checkin Date'] - df['Snapshot Date']).dt.days
df['WeekDay'] = df['Checkin Date'].dt.day_name()
df['DiscountDiff'] = df['Original Price'] - df['Discount Price']
df['DiscountPerc'] = (df['DiscountDiff'] / df['Original Price']) * 100

output_file_path = "./hotels_data_changed.csv"
df.to_csv(output_file_path, index=False)

print(df.head())

## step 2 - Best discount code

## step 3 - Clustering Based on price polices 



Identify the 150 hotels with the most data in the dataset and extract their records.

In [ ]:
import pandas as pd

file_path = "./hotels_data_changed.csv"  
df = pd.read_csv(file_path)

hotel_counts = df['Hotel Name'].value_counts()
top_150_hotels = hotel_counts.head(150).index
filtered_df = df[df['Hotel Name'].isin(top_150_hotels)]

filtered_df.head()

Find the 40 most common check-in dates  in the dataset and extract their records.


In [ ]:
checkin_counts = filtered_df['Checkin Date'].value_counts()
top_40_checkin_dates = checkin_counts.head(40).index
filtered_checkin_df = filtered_df[filtered_df['Checkin Date'].isin(top_40_checkin_dates)]

filtered_checkin_df.head()

## step 4 - Building a Predictive Model for Hotel Pricing Dynamics Using Snapshot Data



## step 5 - PySpark & Mllib for step 3